<a href="#Overview"></a>
# Overview
* <a href="#4c8dfb2f-2382-4144-93e7-e9da792a7034">Week 9 - Imaging and anatomy</a>
  * <a href="#f7c07563-1430-4288-bbdb-32224401f4d8">Fun in Three Dimensions</a>
  * <a href="#7cf740fc-f894-4484-a0f2-83ddeae677b3">Introduction</a>
  * <a href="#03520a2d-f1da-49ea-a66f-7168e52becad">Loading the data, and exploring nibabel</a>
    * <a href="#bbde4f95-8e34-4d83-a417-489125695d41">So the volumes are both 'Nifti's' -- and it appears that they are 120 by 159 by 102 matrices -- that sounds like an 3D MRI volume to me!</a>
      * <a href="#4b62402e-9483-4a3a-9e60-2f679d1cb6b8">Let take a look at a single slice in the middle of the brain using imshow, a matplotlib version of "plot" for showing a 2D image</a>
  * <a href="#e45c062d-1db5-471d-a701-1d460491e36d">Learning About NIfTIs</a>
* <a href="#c62bf9b4-8fc4-4348-a10e-9d5407062551">NIfTI</a>
  * <a href="#38e45e1c-da90-45b7-9879-8f55759d6642">What is NIfTI?</a>
  * <a href="#8a240e83-5b23-4c26-b63f-2190c1506c1c">NIfTI file format</a>
  * <a href="#04ef3b1d-19cd-43f7-885e-f358b49805aa">DICOM vs NlfTI</a>
    * <a href="#6189a6fd-db18-4063-a7f1-d444e51a5263">But we are not going to bother with DICOMS (today)</a>
  * <a href="#aac4ed9e-b87e-4674-857b-af273894a05f">Back to our Nibabel Nifti object --</a>
* <a href="#5cc1947a-569a-4c15-b2ad-3d2dd1766bf4">Let's pull these out!</a>
  * <a href="#3cc01d52-0ff1-4dcc-9d3d-e8af64d04cdc">Exercise 1: fix overlaid segmentation</a>
  * <a href="#a4edc731-b09e-4381-a58b-9cf99ca7668e">Excercise 2 - Excise Neocortex</a>
  * <a href="#2b9e9b8f-528a-47fa-9769-4f5c4200549a">Excercise 3 - Create individual tissue segmentations</a>
  * <a href="#d0bc3d65-621c-451a-869c-11d1d9721716">Excercise 4 -- Euclidean Distance Transforms</a>
* <a href="#cdc66cf9-ba23-4038-bea0-330474d728a9">Huzzah!</a>
  * <a href="#9e24bccb-5f35-4341-863e-c3f12a4c0377">Now we have what we have always dreamed of, a white matter mask that has information regarding its distance to cortex.</a>
  * <a href="#774eeef7-82cb-491c-9e8b-ab716a57be8b">Excercise-ish - Make plotting pretty/easier.</a>
  * <a href="#a23a9851-f4ae-49cd-b10c-98fc97ebf65c">Extra Excercise - Mid-cortical Thickness</a>

<a id="4c8dfb2f-2382-4144-93e7-e9da792a7034"></a>
# Week 9 - Imaging and anatomy
<a href="#Overview">Return to overview</a>

<a id="f7c07563-1430-4288-bbdb-32224401f4d8"></a>
## Fun in Three Dimensions
<a href="#Overview">Return to overview</a>


<a id="7cf740fc-f894-4484-a0f2-83ddeae677b3"></a>
## Introduction
<a href="#Overview">Return to overview</a>

Today we're going to play around with some imaging data, and use some masking techniques to analyze tissue white matter intensity in a more anitomcally relevant way -- as a function of the distance from the surrounding grey matter. Often, groups will average large regions of white matter in analyses which neglects potentially significant spatial differences in organization. I am particularly interested in fetal and early infant brain development, and we can see that the spatial white matter characteristics vary across development. 

<td><img src="wm_devel.jpg" /></td>

<td><img src="gyri.jpg" /></td>

In addition to the specifics of dealing with imaging data/3 dimensional matrices, this will be an opportunity to become familiar with some anatomy and do a bit of light linear algebra.

Main objectives

1. Reading data from NIfTI-format files
    - This will serve you well in when working with any imaging modality: MR, CT, US, etc.  

2. Showing 2D slices (images) and creating a scrollable GUI
    - How do we view a 3D volume without rendering? (Slices!)

3. Masking out specific regions of interest the anatomical image for focused analysis. 
    - This requires aligning the raw data with known anatomical landmarks, often accomplished with registrations to labeled
      atlases. 
    - Masking allows us to single out tissues, and make powerful calculations important for image processing.

4. Measuring 3 dimensional distances.
    - A simple Euclidean Distance Transform can be an incredibly useful tool for image analysis.  

5. Some plot prettifying techniques
    - Python is a really great way to consistently plot data in visually appealing ways. We will take advantage of this!

This software runs directly from your python interpreter, but the first thing we'll need to do is install some packages.

If you haven't already installed nibabel and pyqt, do it now:

    conda install nibabel pyqt
    


In [ ]:
#First we will nee to import some libraries
import os
import scipy
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

<a id="03520a2d-f1da-49ea-a66f-7168e52becad"></a>
## Loading the data, and exploring nibabel
<a href="#Overview">Return to overview</a>

The brain volume and corresponding tissue segmentation are stored in the directory "data"
Lets make path variables for them 

In [ ]:
T2brainpath = os.path.join('data', 'ONPRC18_T2W_brain.nii.gz')
orig_segpath = os.path.join('data', 'growth_label_onprc.nii.gz')

Now we are going to load them into python with the help of nibabel -- do you see anything that might look promising?

In [ ]:
nib?


What helpful documentation! Load in the brain volume and corresponding segmentation, and print() the brain!

In [ ]:
img1 = nib.load(T2brainpath)
img2 = nib.load(orig_segpath)
print(img1)

Interesting... what the heck did we just load in?

In [ ]:
print(type(img1))
print(type(img2))
print(img1.shape)
print(img2.shape)

<a id="bbde4f95-8e34-4d83-a417-489125695d41"></a>
### So the volumes are both 'Nifti's' -- and it appears that they are 120 by 159 by 102 matrices -- that sounds like an 3D MRI volume to me!
<a href="#Overview">Return to overview</a>


So they are both 3D matrices of the same shape (120 by 159 by 102) units, or 'voxels' 
If the voxel size is 0.5mm isotropic (or 0.5mm on all sides) how big is our volume (in this case the FOV for the image) in real world units?

In [ ]:
120*159*102*0.5/1000
#So its about a liter! That seems reasonable 

<a id="4b62402e-9483-4a3a-9e60-2f679d1cb6b8"></a>
#### Let take a look at a single slice in the middle of the brain using imshow, a matplotlib version of "plot" for showing a 2D image
<a href="#Overview">Return to overview</a>


In [ ]:
plt.imshow(img1[:,:,50],cmap='gray')


Uh oh, it looks like that wont work, but we find a helpful hint for how to actually do this!
But first, lets figure out what these niftis atually are

<a id="e45c062d-1db5-471d-a701-1d460491e36d"></a>
## Learning About NIfTIs
<a href="#Overview">Return to overview</a>




This information is adapted from https://www.tasq.ai/glossary/nifti/ and https://nipy.org/nibabel/coordinate_systems.html

<a id="c62bf9b4-8fc4-4348-a10e-9d5407062551"></a>
# NIfTI
<a href="#Overview">Return to overview</a>
<a id="38e45e1c-da90-45b7-9879-8f55759d6642"></a>
## What is NIfTI?
<a href="#Overview">Return to overview</a>
MRI Data (and other imaging modalities like CT, etc.), may be saved in a special file format called NIfTI (Neuroimaging Informatics Technology Initiative). The discipline of medical imaging, as well as brain research and other image-based medical specialties including radiation oncology and psychiatry, make extensive use of it.

The NIfTI format was created to improve upon the shortcomings of the previous standard for storing medical imaging data, the Analyze format.
Along with the actual picture data, NIfTI files also include metadata such as the image’s measurements, voxel size, and direction, as well as the results of any pre-processing that may have been performed on the raw data.

For example, common viewers are all capable of reading NIfTI files (which end in .nii or .nii.gz).

<a id="8a240e83-5b23-4c26-b63f-2190c1506c1c"></a>
## NIfTI file format
<a href="#Overview">Return to overview</a>
There are two components to a NIfTI file:

* the actual image data 
* header information 

_Both the picture data and the header information are kept as binary and text files, respectively._

Some of the fields that make up a NIfTI header format are as follows:

* Dimensions– The number of slices in the z-dimension is an example of a dimension. Dimensions are the total number of voxels in the picture.
* Resolution– Millimeter values for x, y, and z dimensions define the spatial resolution of a 3D model.
* Data- The image’s data type, which might be an unsigned 8-bit integer or a 32-bit floating point number.
* Orientation– Picture orientation refers to the overall orientation of an image, including the x, y, and z axes.
* Intensity– If the data has been scaled in any manner, this is the scale factor for the intensity values.
* Time– If the picture is a time series, the time step is the interval between frames.
* History– An account of the data’s past processing, including any smoothing or normalization that may have occurred.
* Data offset– indicates the position in bytes from which the picture data begins to be read.
* Extra fields– These are fields used to store data unique to particular picture formats, such as those used in diffusion-weighted images, or to record data that is unique to certain applications.

<a id="04ef3b1d-19cd-43f7-885e-f358b49805aa"></a>
## DICOM vs NlfTI
<a href="#Overview">Return to overview</a>
File formats for medical imaging data such as DICOM and NIfTI are similar yet serve distinct functions. NIfTI is mostly employed in the fields of medical imaging and neuroscience research, whereas DICOM is the standard file format for storing and distributing medical pictures.

There are numerous medical imaging software programs and analytic tools that only work with NIfTI files, therefore converting DICOM files is a regular chore. It is possible to transform DICOM to NIfTI using a number of tools and packages, such as:

        pydicom– Python’s pydicom package may be used to read and write DICOM files and convert them to NIfTI.
        
<a id="6189a6fd-db18-4063-a7f1-d444e51a5263"></a>
### But we are not going to bother with DICOMS (today)
<a href="#Overview">Return to overview</a>

<a id="aac4ed9e-b87e-4674-857b-af273894a05f"></a>
## Back to our Nibabel Nifti object --
<a href="#Overview">Return to overview</a>
_A nibabel (and nipy) image is the association of three things:_

* The image data array: a 3D or 4D array of image data

* An affine array that tells you the position of the image array data in a reference space.

* image metadata (data about the data) describing the image, usually in the form of an image header.

So we know that our nifti's are not just the data, they are a combination of header and image data, as well as an associated affine component
<a id="5cc1947a-569a-4c15-b2ad-3d2dd1766bf4"></a>
# Let's pull these out!
<a href="#Overview">Return to overview</a>


In [ ]:
data = img1.get_fdata()
header = img1.header
afftrans = img1.affine 
print(data)
print(header)
print(afftrans)

A problem I often run into is that the preview of a brain volume, ususally a very large matrix with surronding 0's, is not helpful.
Sometimes, I just want a big ol' ugly print out of a bunch of numbers (a slice of the image). We can use _with_ to accomplish this for a unique instance by only momentarily changing some numpy print option parameters.

In [ ]:
with np.printoptions(threshold=np.inf):
    print(data[:,:,50])

Now, try to show a slice again, this time referencing the data object

In [ ]:
plt.imshow(data[:,:,50],cmap='gray')

Great, now overlay the same slice with the corresponding slice of the segmentation, using the colormap 'tab20'

In [ ]:
T1_vol=img1.get_fdata().copy()
segmented_vol=img2.get_fdata().copy()

Here's what the segmenation looks like on its own. Notice the effect of changing the colormap

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,6))
ax[0].imshow(segmented_vol[:,:,50],cmap='gray')
ax[1].imshow(segmented_vol[:,:,50], alpha = 1, cmap ='tab20')

We can use the `alpha` parameter to superimpose a 

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(T1_vol[:,:,50],cmap='gray')
ax.imshow(segmented_vol[:,:,50], alpha = .1, cmap ='tab20')

<a id="3cc01d52-0ff1-4dcc-9d3d-e8af64d04cdc"></a>
## Exercise 1: fix overlaid segmentation
<a href="#Overview">Return to overview</a>

The values for the segmentation are not well distributed, with subcortical grey matter values arbitrarily very high
Create a new segmentation `segmented_vol2` with all values higher than 10 set to 10, and overlay it. Hint: start by setting `segmented_vol2 = segmented_vol` and then thresholding

In [ ]:
segmented_vol2=img2.get_fdata().copy()
%load "answers/answer_001.txt"

Nice! What a clean(ish) segementation! 
There is not an easy way (that I know of) to view 3D images as a series of 2D slices, but the below code will let us see the full 3D volume

In [ ]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

%matplotlib qt 
fig, ax = plt.subplots(1, 1)

tracker = IndexTracker(ax, data)


fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

<a id="a4edc731-b09e-4381-a58b-9cf99ca7668e"></a>
## Excercise 2 - Excise Neocortex
<a href="#Overview">Return to overview</a>
What a groovy looking brain. Ok, Now that we have taken a a good look at our data, lets actually do some business. 
We are interested in neocortical grey and white matter, so the brainstem and cerebellum are going to be a problem. 
Create a new segmentation called neoseg, that includes everything except the cerebellum and brainstem, which are coded as 7s and 8s in the original segmentation.

In [ ]:
neoseg = img2.get_fdata().copy()
%load "answers/answer_002.txt"

<a id="2b9e9b8f-528a-47fa-9769-4f5c4200549a"></a>
## Excercise 3 - Create individual tissue segmentations
<a href="#Overview">Return to overview</a>
Stellar work, yall. 
So, now what we want is to create a new map of the white matter tissue for the whole brain, where each voxel's value is the distance to the nearest non-white matter voxel. 
Easy -- to do this we only need 3ish things.
 First, initialize temporary volumes filled with zeros the same size as out brain/segmentation volumes, and name them gm and wm and csf (for later). 
 _Hint: newvol=volofchoice*0_

In [ ]:
%load "answers/answer_003.txt"

Great, now put 1s in each matrix corresponding to the original segmentation for each of these new volumes -- HINT! In the original segmentation, or your new segpar variable, CSF corresponds to 1, GM corresponds to 2, and WM corresponds to 3!


In [ ]:
%load "answers/answer_004.txt"

<a id="d0bc3d65-621c-451a-869c-11d1d9721716"></a>
## Excercise 4 -- Euclidean Distance Transforms
<a href="#Overview">Return to overview</a>
Now that we have these volumes, we can get tricky with calculating distances. 
Remember, we want white matter to not be a binary "1" denoting yes this tissue is in fact white matter, but instead denote its location in a bank of white matter, in other words, that white matter voxels distance from the closest non-white matter voxel.


Thankfully scipy has a function _ndimage.distance_transform_edt_ that allows one to caclulate the exact Euclidean distance transform, which calculates the distance transform of the input, by replacing each foreground (non-zero) element, with its shortest distance to the background (any zero-valued element).

Suppose we have the binary image below, what does the distance transform look like? Try for yourself and label it bindist -- Hint: the function takes the argument 'f(array==value)'

In [ ]:
binary_image = np.array([[0,0,1,0,0],
                         [0,1,1,1,0],
                         [0,1,1,1,0],
                         [0,0,1,0,0],])

In [ ]:
%load "answers/answer_005.txt"

Awesome, does that look like you would expect?


Now we can do the same thing but for our binary tissue maps!!
Create a new volume called wmdist that represents very voxels distance from the grey matter, then SHOW THAT PUPPY.

In [ ]:
%load "answers/answer_006.txt"

In [ ]:
#To help with showing
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(wmdist[:,:,50],cmap='gray')

Heck yeah! So, now we have every non-GM voxel's distance from GM. 
But, I think we should save the vignette for instagram... I dont think we care very much about knowing how far away we are from cortex in CSF. 

We can get rid of all that by masking out all the voxels that arent white matter to zero. 

_Any thoughts?_

In [ ]:
%load "answers/answer_007.txt"

In [ ]:
#To help with showing
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(wmdist[:,:,50],cmap='gray')

<a id="cdc66cf9-ba23-4038-bea0-330474d728a9"></a>
# Huzzah!
<a href="#Overview">Return to overview</a>
<a id="9e24bccb-5f35-4341-863e-c3f12a4c0377"></a>
## Now we have what we have always dreamed of, a white matter mask that has information regarding its distance to cortex.
<a href="#Overview">Return to overview</a>


Lets zoom into the left inferior temporal gyrus (ITG) to get a closer look at what we just accomplished.  

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
left_ITG=data[95:120,75:110,50]
left_ITG_dist=wmdist[95:120,75:110,50]
ax.imshow(left_ITG,cmap='gray')
ax.imshow(left_ITG_dist,cmap='viridis', alpha = .2)

So now we just need to get an account of every one of these voxel's "distance values" and the corresponding image intensity values, and then mean the intensity values by unique distance value. 

In [ ]:
#Let's get all distances ignoring 0's
distances = wmdist[wmdist > 0]
#y then gets all those overlapping image voxels that arent at 0 distance
intensities = T1_vol[wmdist > 0]

#using np.unique, we can get all the unique values of our array,
#while also getting a  also return the indices of the unique array
#that can be used to reconstruct our array -- this is needed to be able to
# average all of the intensity values subsequently
distances_new,inv,cnt = np.unique(distances,return_inverse=True,return_counts=True)
# now we feed bincount the indices from above (inv, and give our intensity values
# as the weights, and divide by the count for each unique index!!

intensities_new = np.bincount(inv,weights=intensities)/cnt
#Here is another way to accomplish the same thing uising an ordered dictionary
#from collections import OrderedDict
#values = OrderedDict()
#for v1, v2 in zip(x, y):
#    values.setdefault(v1, []).append(v2)

#new_x = list(values.keys())
#new_y = [sum(values[i]) / len(values[i]) for i in values]



Alright team, nice work -- now how many voxels are we querrying?
Once you figure that out, plot the distances_new and intensities_new out to 5mm!!

In [ ]:
%load "answers/answer_008.txt"

 Neato, we can see that intially, white matter gets darker to a minimum a little more than one mm away from cortex. This corresponds to the center of your average fasicle. However, we then see an increase in intensity at greater differences. 
Why do you think this is? (Think about the internal capsule, imperfect segmentations, and the ridiculousness of trying to average every WM voxel across the brain! ...But we had fun

<a id="774eeef7-82cb-491c-9e8b-ab716a57be8b"></a>
## Excercise-ish - Make plotting pretty/easier.
<a href="#Overview">Return to overview</a>
That is one ugly plot -- lets have fun making it pretty. Starting with RC params!
Customizing Matplotlib with style sheets and rcParams
Tips for customizing the properties and default styles of Matplotlib.

There are three ways to customize Matplotlib:

Setting rcParams at runtime.

Using style sheets.

Changing your matplotlibrc file.

Setting rcParams at runtime takes precedence over style sheets, style sheets take precedence over matplotlibrc files.

In [ ]:
import matplotlib as mpl

SMALL_SIZE = 30
MEDIUM_SIZE = 30
BIGGER_SIZE = 40

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
mpl.rcParams['font.family'] = 'Arial'

In [ ]:
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

#Intitialize Figure
fig, ax = plt.subplots(figsize=(12,8))
#Make a fun arbitrary conditional color scheme
wackycolors = np.where(distances_new<3,'peachpuff',np.where(intensities_new<2.4,'lightseagreen','rebeccapurple'))
#Plot that puppy
plt.scatter(distances_new, intensities_new, marker = '^', color=wackycolors)
#Set the x limit to 10
plt.xlim(0,10)
#Unneccesarily rotate the x tick labels
plt.xticks(rotation = 45, ha = 'right', rotation_mode = 'anchor')
#Title plot as a compliment, and move it to the inside of your plot, and shift to the left a bit to leave room for the legend
plt.title("Cool Graph Bro", x=0.40, y = 1, pad = -28)
#Axis labels
plt.ylabel("T2 Intensity (Arbitrary)")
plt.xlabel("Distance from Cortex (mm)" )
#Create your custom legend, because we can!
#Use mpatches for bar type plots, and Line2D for line/scatter plots
custom_lines = [mpatches.Patch(facecolor='pink', edgecolor='k'),
                Line2D([0], [0], marker = '^', color="black", lw=0,markersize=15),
               Line2D([0], [0], color='red', linewidth=1.5)]
plt.legend(custom_lines, ["Pink Patch","Black Triangle Point",'Red$_{Line}$' ],fontsize=20)

<a id="a23a9851-f4ae-49cd-b10c-98fc97ebf65c"></a>
## Extra Excercise - Mid-cortical Thickness
<a href="#Overview">Return to overview</a>


OK, if we have time we will make a mid cortical thickness mask which is super useful for any kind of surface based analysis. 

Now the thinky bit -- we want to get the distance for outside voxels into the brain and only wm voxels out of the brain... how do we do this?
We need 2 volumes, 'inside' and 'outside' with 1s for tissue in the brain (inside) and 1s for voxels out of the brain (outside) -- To do this, we need to make a combination volume of wm and gm (1's for both, everything else 0) called inside, and then create a mirror image (0's for both, everything else 1's)
Then create volumes with the distance tranform for the outside labelled dist2o and wm labelled dist2i  

In [ ]:
%load "answers/answer_009.txt"

In [ ]:
#look at aeverything
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(inside[:,:,50],cmap='gray')
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(outside[:,:,50],cmap='gray')
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(dist2o[:,:,50],cmap='gray')
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(dist2i[:,:,50],cmap='gray')

Now we have 2 volumes dist2o and dist2i which we can use to get a middle segmentation of the grey matter. The middle boundary of the grey matter is where the distance of the outside of the brain to the inside is equal to the distance of the wm to the outside of the brain. BUT, we just want a ~1 voxel thick border so we will get all voxels where dist2o is smaller than dist2i, and then make a new volume midvox that is the distance transform of the the voxels just to the outside

In [ ]:
midseg=wm.astype(np.int16)*0
midseg[dist2o<dist2i]=1;
inmidseg = abs(midseg-1)
midvox=wm.astype(np.int16)*0
midvox[scipy.ndimage.distance_transform_edt(inmidseg)==1]=1
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(midseg[:,:,45])
fig, ax = plt.subplots(figsize=(12,8))
testseg = neoseg.astype(np.int16)
testseg[testseg>5]=0
ax.imshow(testseg[:,:,45])
ax.imshow(midvox[:,:,45], alpha = .1,cmap='gray')